# IDENTIFYING DISASTER IN AERIAL IMAGERY
This solution builds an aerial image classification model which when embedded on a drone can identify calamities such as collapsed buildings, flood, or fire with high confidence. This solution can  autonomously monitor a disaster-stricken area and alert in real-time.

## Why is this important?
Government authorities face following challenges at the event of a disaster :
- Poor intersectoral coordination
- Lack of an early warning system
- Slow response from the relief agencies
- Lack of trained / dedicated search and rescue teams

## What impact can it have?
- Governments can coordinate better disaster relief programs.
- Enhanced situational awareness
- Access to early warnings and verified reports in real-time.
- Capable of operating in remote and difficult to access areas.
- Faster mitigation of impact on environment and human population
- Protection of life and property from prolonged damage
- Deloitte can pitch this solution to GPS industry 
to help government manage disaster response program effectively





In [ ]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
cudnn.benchmark = True
plt.ion()   # interactive mode
torch.manual_seed(17)
from pandas.core.common import flatten
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import glob
from tqdm import tqdm
import random
random.seed(0)
from sklearn.metrics import classification_report
import json
import plotly.io as pio
pio.renderers.default = "png"
from PIL import Image as PilImage
from omnixai.data.image import Image
from omnixai.explainers.vision.specific.gradcam.pytorch.gradcam import GradCAM

#       Image Augmentation

In [ ]:
train_transforms = A.Compose(
    [
        A.Resize(224,224),
        A.OneOf([
            A.IAAAffine(scale=1.0, translate_percent=0.2, translate_px=None, rotate=0.0, shear=0.0, order=1, cval=0, mode='reflect', always_apply=False, p=0.6),
            A.RandomBrightnessContrast(brightness_limit=(-0.1,0.2), contrast_limit=(-0.1, 0.2), p=0.5),
            A.HorizontalFlip(p=0.5),
            A.RandomCrop(width=200, height=200),
            A.CLAHE(p=0.3),
            
            #A.GridDistortion(p=0.3),
            A.SafeRotate(limit=90, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
            ], p=0.75),
        A.Resize(224,224),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        
        
        ToTensorV2(),
    ]
)

test_transforms = A.Compose(
    [
        A.Resize(224,224),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)

#       Create Train, Valid and Test sets

In [ ]:
train_data_path = 'dataset/train/' 
train_image_paths = [] #to store image paths in list
classes = [] #to store class values


# get all the paths from train_data_path and append image paths and class to to respective lists

for data_path in glob.glob(train_data_path + '/*'):
    classes.append(data_path.split('/')[-1]) 
    train_image_paths.append(glob.glob(data_path + '/*'))
    
for data_path in glob.glob(val_data_path + '/*'):
    train_image_paths.append(glob.glob(data_path + '/*'))
     
    
train_image_paths = list(flatten(train_image_paths))
random.shuffle(train_image_paths)

print('train_image_path example: ', train_image_paths[0])
print('class example: ', classes[0])

# split train valid test from train paths (70,20, 10)
train_image_paths, valid_image_paths, test_image_paths = train_image_paths[:int(0.7*len(train_image_paths))],train_image_paths[int(0.7*len(train_image_paths)):int(0.9*len(train_image_paths))],train_image_paths[int(0.9*len(train_image_paths)):]
test_image_paths = list(flatten(test_image_paths))
print("Train size: {}\nValid size: {}\nTest size: {}".format(len(train_image_paths), len(valid_image_paths), len(test_image_paths)))

In [ ]:
#######################################################
#      Create dictionary for class indexes
#######################################################
idx_to_class = {i:j for i, j in enumerate(sorted(classes))}
class_to_idx = {value:key for key,value in idx_to_class.items()}
idx_to_class

#  Define Dataloader

In [ ]:
class DisasterDataset(Dataset):
    def __init__(self, image_paths, transform=False):
        self.image_paths = image_paths
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_filepath = self.image_paths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        label = image_filepath.split('/')[-2]
        label = class_to_idx[label]
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        
        return image, label
    


train_dataset = DisasterDataset(train_image_paths,train_transforms)
valid_dataset = DisasterDataset(valid_image_paths,train_transforms) #test transforms are applied
test_dataset = DisasterDataset(test_image_paths,test_transforms)

image_datasets=dict()
image_datasets['train'] = train_dataset
image_datasets['val'] = valid_dataset
image_datasets['test'] = test_dataset

In [ ]:

input_size=224
batch_size = 8

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#######################################################
#                  Visualize Dataset
#         Images are plotted after augmentation
#######################################################

def visualize_augmentations(dataset, idx=0, samples=10, cols=5, random_img = False):
    
    dataset = copy.deepcopy(dataset)
    #we remove the normalize and tensor conversion from our augmentation pipeline
    dataset.transform = A.Compose([t for t in dataset.transform if not isinstance(t, (A.Normalize, ToTensorV2))])
    rows = samples // cols
    
        
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 8))
    for i in range(samples):
        if random_img:
            idx = np.random.randint(1,len(train_image_paths))
        image, lab = dataset[idx]
        ax.ravel()[i].imshow(image)
        ax.ravel()[i].set_axis_off()
        ax.ravel()[i].set_title(idx_to_class[lab])
    plt.tight_layout(pad=1)
    plt.show()    

visualize_augmentations(train_dataset,np.random.randint(1,len(train_image_paths)), random_img = True)



# Define Training Function

In [ ]:
#######################################################
#                  Train function
#            to learn and update weights
#######################################################

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    # keeping-track-of-losses 
    train_losses = []
    valid_losses = []
    train_acc = []
    valid_acc = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss =  100

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
      
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                
            

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            if phase == 'train':
                scheduler.step(epoch_loss)
                train_losses.append(epoch_loss)
                train_acc.append(epoch_acc.detach().cpu().numpy() )
                
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                valid_losses.append(epoch_loss)
                valid_acc.append(epoch_acc.detach().cpu().numpy() )

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_loss:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)


    return model, train_losses, valid_losses, train_acc, valid_acc

In [ ]:
##########################################################
#        load resnet18 backbone and add a new dense layer
##########################################################

model_ft = models.resnet18(weights=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 4)

In [ ]:
#Define Loss function, Optimizer and Learning Rate
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.00001, momentum=0.9)
# Find learning rate in a cyclic manner
exp_lr_scheduler = lr_scheduler.CyclicLR(optimizer_ft, base_lr=0.00001, max_lr=0.01)


# Start Training

In [ ]:

model_ft, train_losses, valid_losses, train_acc, valid_acc = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=200)


In [ ]:
##########################################################
#                 visualize loss vs epochs
##########################################################
    
    %matplotlib inline
    %config InlineBackend.figure_format = 'retina'
    
    plt.plot(train_losses, label='Training loss')
    plt.plot(valid_losses, label='Validation loss')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend(frameon=False)


In [ ]:
##########################################################
#                 visualize accuracy vs epochs
##########################################################
    
    
    
    plt.plot(train_acc, label='Training Accuracy')
    plt.plot(valid_acc, label='Validation Accuracy')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend(frameon=False)

# Evaluate on Test Data

In [ ]:
##########################################################
#               Evaluate model on test data
##########################################################

dl_test = torch.utils.data.DataLoader(image_datasets['test'], batch_size=dataset_sizes['test'],
                                             shuffle=False, num_workers=4)

# Get a batch of test data
true = []
pred = []
for i in  range(1):
    test_input , classes= next(iter(dl_test))
    true.extend(classes.tolist())
    test_input = test_input.to(device)
    outputs = model_ft(test_input)
    _, predicted = torch.max(outputs, 1)

    probability =  F.softmax(outputs, dim=1)

    top_probability, top_class = probability.topk(1, dim=1)

    predicted = predicted.cpu().detach().numpy()
    pred.extend(predicted)
    predicted = predicted.tolist()[0]

    label = idx_to_class[predicted]


In [ ]:
##########################################################
#             Generate Classification Report
##########################################################

target_names = ['collapsed_building', 'fire', 'flood', 'normal']
print(classification_report(true, pred, target_names = target_names))

# Save and Load Model

This is to save the model once trained and load the trained model for future evalution

In [ ]:
##########################################################
#                     Save  model
##########################################################
torch.save(model_ft, 'trained_model.pth')

In [ ]:
##########################################################
#                   Load  saved  model
##########################################################
model_ft = torch.load('trained_model.pth')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft.to(device)